# Adverserial Attack Evaluation
After normal training and adversarial training we evaluate all of our models on their robustness. We test them on four different attacks, namely

- a custom attack
- BAE attack from textattack
- Textfooler attack from textattack



Following naming will be used below:
- <strong>Pre-Trained Model:</strong> This is the [RoBERTa model ](https://huggingface.co/docs/transformers/model_doc/roberta) model from Huggingface
- <strong>Initial Hate Speech Model:</strong> This is our RoBERTa model, which we trained on the Hate speech data set.
- <strong>Trained Hate Speech Model:</strong> RoBERTa model, which was trained using adversarial training
    - <strong>Custom Trained:</strong> RoBERTa model, which was trained using the custom attack
    - <strong>Custom Finetuned:</strong> Initial Hate Speech Model, which was finetuned using the custom attack
    - <strong>BAE Trained:</strong> RoBERTa model, which was trained using the BAE attack


## Install

In [ ]:
# The kernel needs to be restarted after the pip installs
# It appears that textattack has dependency conflicts if not run on an ARM chip.
!pip3 install transformers
!pip3 install textattack
!pip3 install --force-reinstall textattack # force is often needed due to dependency conflicts
!pip3 install --upgrade tensorflow
!pip3 install sentence_transformers

## Import

In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.wsd import lesk
nltk.download('stopwords')
nltk.download('punkt')

# textattack packages
import textattack
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.semantics import WordEmbeddingDistance

# transformers packages
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
from transformers import RobertaTokenizer, RobertaForSequenceClassification

from torch.utils.data import DataLoader, TensorDataset

from tqdm import tqdm

from trainer import Trainer


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/simonzollhoefer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/simonzollhoefer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!




#### Data cleaning
Since the data needs to be cleaned for the attack, we defined the following function.

In [ ]:
#this is copy from https://www.kaggle.com/code/soumyakushwaha/ethicalcommunicationai
# ----------------------------------------
stopword = set(stopwords.words('english'))

def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub(r"\@w+|\#",'',text)
    text = re.sub(r"[^\w\s]",'',text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    tweet_tokens = word_tokenize(text)
    filtered_tweets=[w for w in tweet_tokens if not w in stopword] #removing stopwords
    return " ".join(filtered_tweets)
#--------------------------------------------------------------------------------------

#### Load Dataset


In [13]:
# Constants
SEED = 42
BATCH_SIZE = 32
LEARNING_RATE = 1e-5
MAX_TEXT_LENGTH = 512
EPOCHS = 10
MODEL_PATH = 'roberta_model.bin'
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Set seeds
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)


labeled_data = pd.read_csv('./datasets/hate_speech_data.csv')
# Hate Speech and Offensive Language Data: 25.3k total entries.
# - Class 0: 1,430 entries (hate speech)
# - Class 1: 19,190 entries (offensive language)
# - Class 2: 4,163 entries (neither)

# Processing labeled hate speech dataset
hate_offensive_data = labeled_data[labeled_data['class'] != 2].copy()
hate_offensive_data.loc[:, 'category'] = hate_offensive_data['class'].replace([0, 1], 1)
hate_offensive_data = hate_offensive_data.rename(columns={'tweet': 'text'})

# Test 1 ---
# Select data for each class
hate_speech_data = labeled_data[labeled_data['class'] == 0].copy()
offensive_data = labeled_data[labeled_data['class'] == 1].copy()
neither_data = labeled_data[labeled_data['class'] == 2].copy()
sample_size = len(hate_speech_data)
offensive_sample = offensive_data.sample(n=sample_size, random_state=SEED)
neither_sample = neither_data.sample(n=sample_size, random_state=SEED)
hate_speech_data['category'] = 1
offensive_sample['category'] = 1
neither_sample['category'] = 0
sampled_data = pd.concat([hate_speech_data, offensive_sample, neither_sample], ignore_index=True)[['tweet', 'category']]
sampled_data.rename(columns={'tweet': 'text', 'category': 'label'}, inplace=True)
sampled_data['text'] = sampled_data['text'].apply(clean_text)  # Assuming clean_text is a defined function
train_data, intermediate_data = train_test_split(sampled_data, test_size=0.3, random_state=SEED)
validation_data, test_data = train_test_split(intermediate_data, test_size=0.5, random_state=SEED)
train_tokens = tokenizer(train_data['text'].tolist(), padding=True, truncation=True, max_length=MAX_TEXT_LENGTH, return_tensors='pt')
validation_tokens = tokenizer(validation_data['text'].tolist(), padding=True, truncation=True, max_length=MAX_TEXT_LENGTH, return_tensors='pt')
test_tokens = tokenizer(test_data['text'].tolist(), padding=True, truncation=True, max_length=MAX_TEXT_LENGTH, return_tensors='pt')
print(f"New Train data shape: {train_data.shape}")
print(f"New Validation data shape: {validation_data.shape}")
print(f"New Test data shape: {test_data.shape}")


New Train data shape: (3003, 2)
New Validation data shape: (643, 2)
New Test data shape: (644, 2)


#### Method to load models

In [14]:
def load_model(file_name):
    config = RobertaConfig()
    config.num_labels = 2
    roberta_base_config = {
      "architectures": [
        "RobertaForMaskedLM"
      ],
      "attention_probs_dropout_prob": 0.1,
      "bos_token_id": 0,
      "eos_token_id": 2,
      "hidden_act": "gelu",
      "hidden_dropout_prob": 0.1,
      "hidden_size": 768,
      "initializer_range": 0.02,
      "intermediate_size": 3072,
      "layer_norm_eps": 1e-05,
      "max_position_embeddings": 514,
      "model_type": "roberta",
      "num_attention_heads": 12,
      "num_hidden_layers": 12,
      "pad_token_id": 1,
      "type_vocab_size": 1,
      "vocab_size": 50265
    }

    for key in roberta_base_config.keys():
        setattr(config, key, roberta_base_config[key])

    model = RobertaForSequenceClassification(config)
    map_location=torch.device('cpu')
    model.load_state_dict(torch.load(file_name, map_location=map_location))
    tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
    model.eval()
    model.to(map_location)
    return model, tokenizer

## Attack Setup
After loading the initial models, we can attack it. To do so we try different attacks:

- a custom attack
- Bae attack from textattack
- TextFooler attack from textattack



### Custom Attacker

In [ ]:
ATTACK_SEED = 71

def create_custom_attacker(model_wrapper, dataset, num_examples=20, nr=0):

    # Define custom attack based on https://textattack.readthedocs.io/en/latest/api/attack.html used for training loop

    #UntagetedClassification: An untargeted attack on classification models which attempts
    #to minimize the score of the correct label until it is no longer the predicted label.
    goal_function = textattack.goal_functions.UntargetedClassification(model_wrapper)

    constraints = [
        RepeatModification(), # prevents the same word from being modified multiple times
        StopwordModification(), # controls the modification of stopwords (e.g., "the," "is," "and")
        WordEmbeddingDistance(min_cos_sim=0.9), # measures the cosine similarity between word embeddings to ensure that the replacement word is semantically similar
    ]

    transformation = textattack.transformations.word_swaps.word_swap_embedding.WordSwapEmbedding(max_candidates=50) # (50 is default)
    search_method = textattack.search_methods.GreedyWordSwapWIR(wir_method="delete")
    custom_attack = textattack.Attack(goal_function, constraints, transformation, search_method) # perform the attack

    attack_args = textattack.AttackArgs(num_examples=num_examples, log_to_csv=f"log_{nr}.csv", checkpoint_interval=5, checkpoint_dir="checkpoints", disable_stdout=True)
    custom_attacker = textattack.Attacker(custom_attack, dataset, attack_args)

    return custom_attacker


### BAE Attacker

In [ ]:
def create_bae_attacker(model_wrapper, dataset, num_examples=20, nr=0):

    bae_attack = textattack.attack_recipes.bae_garg_2019.BAEGarg2019.build(model_wrapper)

    # Attack 20 samples with CSV logging and checkpoint saved every 5 interval
    attack_args = textattack.AttackArgs(num_examples=num_examples, log_to_csv=f"log_{nr}.csv", checkpoint_interval=5, checkpoint_dir="checkpoints", disable_stdout=True)
    bae_attacker = textattack.Attacker(bae_attack, dataset, attack_args)
    return bae_attacker


### Textfooler Attacker

In [ ]:
def create_textfooler_attacker(model_wrapper, dataset, num_examples=20, nr=0):

    textfooler_attack = textattack.attack_recipes.textfooler_jin_2019.TextFoolerJin2019.build(model_wrapper)
    # Attack 20 samples with CSV logging and checkpoint saved every 5 interval
    attack_args = textattack.AttackArgs(num_examples=num_examples, log_to_csv=f"log_{nr}.csv", checkpoint_interval=5, checkpoint_dir="checkpoints", disable_stdout=True)
    textfooler_attacker = textattack.Attacker(textfooler_attack, dataset, attack_args)
    return textfooler_attacker

### BERT Attacker

In [ ]:
def create_bert_attacker(model_wrapper, dataset, num_examples=20, nr=0):

    bert_attack = textattack.attack_recipes.bert_attack_li_2020.BERTAttackLi2020.build(model_wrapper)

    # Attack 20 samples with CSV logging and checkpoint saved every 5 interval
    attack_args = textattack.AttackArgs(num_examples=num_examples, log_to_csv=f"log_{nr}.csv", checkpoint_interval=5, checkpoint_dir="checkpoints", disable_stdout=True, query_budget=100, num_workers_per_device=2)
    bert_attacker = textattack.Attacker(bert_attack, dataset, attack_args)
    return bert_attacker

## Evaluate the Models on Adversarial Attacks


In [ ]:
# global variables

# Run attack with defined dataset
num_examples = 300

sample_size = math.ceil(num_examples/2)

balanced_dataset = validation_data.groupby("label").sample(n=sample_size, random_state=SEED)
balanced_dataset = balanced_dataset.sample(frac=1).reset_index(drop=True)

temp = list(balanced_dataset.itertuples(index=False, name=None))
dataset = textattack.datasets.Dataset(temp)

num_examples = 300

#### Initial Hate Speech Model

As the first step we want to get a baseline of the accuracy under attack of our Initial Hate Speech Model (the training of this model is done in notebook inital_hate_speech_model_training.ipynb).

In [ ]:
initial_hate_speech_model, tokenizer = load_model('initial_hate_speech_model.bin')
initial_hate_speech_model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(initial_hate_speech_model, tokenizer)

In [ ]:
# Custom attack on Initial Hate Speech Model
custom_attacker = create_custom_attacker(initial_hate_speech_model_wrapper, dataset, num_examples=num_examples, nr=0)
custom_attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log_0.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 5 / 0 / 5:   2%|▏         | 5/300 [00:11<11:30,  2.34s/it]textattack: Saving checkpoint under "checkpoints/1698240425872.ta.chkpt" at 2023-10-25 21:27:05 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 9 / 1 / 10:   3%|▎         | 10/300 [00:26<12:45,  2.64s/it]textattack: Saving checkpoint under "checkpoints/1698240440547.ta.chkpt" at 2023-10-25 21:27:20 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 10 / 3 / 15:   5%|▌         | 15/300 [00:31<10:03,  2.12s/it]textattack: Saving checkpoint under "checkpoints/1698240445926.ta.chkpt" at 2023-10-25 21:27:25 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 38 / 214 / 33 / 285:  95%|█████████▌| 285/300 [12:41<00:40,  2.67s/it]textattack: Saving checkpoint under "checkpoints/1698241175786.ta.chkpt" at 2023-10-25 21:39:35 after 285 attacks.


[Succeeded / Failed / Skipped / Total] 39 / 218 / 33 / 290:  97%|█████████▋| 290/300 [12:57<00:26,  2.68s/it]textattack: Saving checkpoint under "checkpoints/1698241191438.ta.chkpt" at 2023-10-25 21:39:51 after 290 attacks.


[Succeeded / Failed / Skipped / Total] 39 / 223 / 33 / 295:  98%|█████████▊| 295/300 [13:15<00:13,  2.70s/it]textattack: Saving checkpoint under "checkpoints/1698241209637.ta.chkpt" at 2023-10-25 21:40:09 after 295 attacks.


[Succeeded / Failed / Skipped / Total] 39 / 227 / 34 / 300: 100%|██████████| 300/300 [13:24<00:00,  2.68s/it]textattack: Saving checkpoint under "checkpoints/1698241218206.ta.chkpt" at 2023-10-25 21:40:18 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 39 / 227 / 34 / 300: 100%|██████████| 300/300 [13:24<00:00,  2.68s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 39     |
| Number of failed attacks:     | 227    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 75.67% |
| Attack success rate:          | 14.66% |
| Average perturbed word %:     | 21.59% |
| Average num. words per input: | 8.53   |
| Avg num queries:              | 18.17  |
+-------------------------------+--------+


In [ ]:
# BAE attack on Initial Hate Speech Model
bae_attacker = create_bae_attacker(initial_hate_speech_model_wrapper, dataset, num_examples=num_examples, nr=1)
bae_attacker.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log_1.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 78 / 7 / 15 / 100:  33%|███▎      | 100/300 [15:20<30:41,  9.21s/it]textattack: Saving checkpoint under "checkpoints/1698244773254.ta.chkpt" at 2023-10-25 22:39:33 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 160 / 14 / 26 / 200:  67%|██████▋   | 200/300 [30:28<15:14,  9.14s/it]textattack: Saving checkpoint under "checkpoints/1698245681460.ta.chkpt" at 2023-10-25 22:54:41 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 244 / 22 / 34 / 300: 100%|██████████| 300/300 [46:41<00:00,  9.34s/it]textattack: Saving checkpoint under "checkpoints/1698246654551.ta.chkpt" at 2023-10-25 23:10:54 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 244 / 22 / 34 / 300: 100%|██████████| 300/300 [46:41<00:00,  9.34s/it]






+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 244    |
| Number of failed attacks:     | 22     |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 7.33%  |
| Attack success rate:          | 91.73% |
| Average perturbed word %:     | 31.7%  |
| Average num. words per input: | 8.53   |
| Avg num queries:              | 66.03  |
+-------------------------------+--------+


In [ ]:
# Textfooler attack on Initial Hate Speech Model
textfooler_attacker = create_textfooler_attacker(initial_hate_speech_model_wrapper, dataset, num_examples=num_examples, nr=2)
textfooler_attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log_2.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 75 / 10 / 15 / 100:  33%|███▎      | 100/300 [17:48<35:36, 10.68s/it]textattack: Saving checkpoint under "checkpoints/1698247725528.ta.chkpt" at 2023-10-25 23:28:45 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 157 / 17 / 26 / 200:  67%|██████▋   | 200/300 [35:24<17:42, 10.62s/it]textattack: Saving checkpoint under "checkpoints/1698248782208.ta.chkpt" at 2023-10-25 23:46:22 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 243 / 23 / 34 / 300: 100%|██████████| 300/300 [51:20<00:00, 10.27s/it]textattack: Saving checkpoint under "checkpoints/1698249738265.ta.chkpt" at 2023-10-26 00:02:18 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 243 / 23 / 34 / 300: 100%|██████████| 300/300 [51:20<00:00, 10.27s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 243    |
| Number of failed attacks:     | 23     |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 7.67%  |
| Attack success rate:          | 91.35% |
| Average perturbed word %:     | 31.06% |
| Average num. words per input: | 8.53   |
| Avg num queries:              | 81.98  |
+-------------------------------+--------+


#### Custom Trained Model

Now we test our trained models and check for their new accurancy under attack. The training of the models is done in the notebook (adversarial_training.ipynb).

In [ ]:
custom_trained_model, tokenizer = load_model('custom_trained_model.bin')
custom_trained_model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(custom_trained_model, tokenizer)

In [ ]:
# Custom attack on Custom Trained Model
custom_attacker = create_custom_attacker(custom_trained_model_wrapper, dataset, num_examples=num_examples, nr=4)
custom_attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log_4.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 5 / 0 / 5:   2%|▏         | 5/300 [00:10<10:46,  2.19s/it]textattack: Saving checkpoint under "checkpoints/1698241231949.ta.chkpt" at 2023-10-25 21:40:31 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 9 / 1 / 10:   3%|▎         | 10/300 [00:25<12:12,  2.53s/it]textattack: Saving checkpoint under "checkpoints/1698241246242.ta.chkpt" at 2023-10-25 21:40:46 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 11 / 2 / 15:   5%|▌         | 15/300 [00:32<10:22,  2.18s/it]textattack: Saving checkpoint under "checkpoints/1698241253731.ta.chkpt" at 2023-10-25 21:40:53 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 30 / 225 / 30 / 285:  95%|█████████▌| 285/300 [13:01<00:41,  2.74s/it]textattack: Saving checkpoint under "checkpoints/1698242002160.ta.chkpt" at 2023-10-25 21:53:22 after 285 attacks.


[Succeeded / Failed / Skipped / Total] 31 / 229 / 30 / 290:  97%|█████████▋| 290/300 [13:17<00:27,  2.75s/it]textattack: Saving checkpoint under "checkpoints/1698242018703.ta.chkpt" at 2023-10-25 21:53:38 after 290 attacks.


[Succeeded / Failed / Skipped / Total] 31 / 234 / 30 / 295:  98%|█████████▊| 295/300 [13:36<00:13,  2.77s/it]textattack: Saving checkpoint under "checkpoints/1698242037289.ta.chkpt" at 2023-10-25 21:53:57 after 295 attacks.


[Succeeded / Failed / Skipped / Total] 32 / 237 / 31 / 300: 100%|██████████| 300/300 [13:45<00:00,  2.75s/it]textattack: Saving checkpoint under "checkpoints/1698242046077.ta.chkpt" at 2023-10-25 21:54:06 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 32 / 237 / 31 / 300: 100%|██████████| 300/300 [13:45<00:00,  2.75s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 32     |
| Number of failed attacks:     | 237    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 79.0%  |
| Attack success rate:          | 11.9%  |
| Average perturbed word %:     | 24.11% |
| Average num. words per input: | 8.53   |
| Avg num queries:              | 18.38  |
+-------------------------------+--------+


In [ ]:
# BAE attack on Custom Trained Model
bae_attacker = create_bae_attacker(custom_trained_model_wrapper, dataset, num_examples=num_examples, nr=5)
bae_attacker.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log_5.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 79 / 8 / 13 / 100:  33%|███▎      | 100/300 [15:35<31:10,  9.35s/it]textattack: Saving checkpoint under "checkpoints/1698250678254.ta.chkpt" at 2023-10-26 00:17:58 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 160 / 15 / 25 / 200:  67%|██████▋   | 200/300 [30:07<15:03,  9.04s/it]textattack: Saving checkpoint under "checkpoints/1698251550509.ta.chkpt" at 2023-10-26 00:32:30 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 245 / 24 / 31 / 300: 100%|██████████| 300/300 [44:55<00:00,  8.98s/it]textattack: Saving checkpoint under "checkpoints/1698252438179.ta.chkpt" at 2023-10-26 00:47:18 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 245 / 24 / 31 / 300: 100%|██████████| 300/300 [44:55<00:00,  8.98s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 245    |
| Number of failed attacks:     | 24     |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 8.0%   |
| Attack success rate:          | 91.08% |
| Average perturbed word %:     | 31.11% |
| Average num. words per input: | 8.53   |
| Avg num queries:              | 65.36  |
+-------------------------------+--------+


In [ ]:
# Textfooler attack on Custom Trained Model
textfooler_attacker = create_textfooler_attacker(custom_trained_model_wrapper, dataset, num_examples=num_examples, nr=6)
textfooler_attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log_6.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 76 / 11 / 13 / 100:  33%|███▎      | 100/300 [18:29<36:58, 11.09s/it]textattack: Saving checkpoint under "checkpoints/1698253547906.ta.chkpt" at 2023-10-26 01:05:47 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 156 / 19 / 25 / 200:  67%|██████▋   | 200/300 [35:06<17:33, 10.53s/it]textattack: Saving checkpoint under "checkpoints/1698254545032.ta.chkpt" at 2023-10-26 01:22:25 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 241 / 28 / 31 / 300: 100%|██████████| 300/300 [52:46<00:00, 10.55s/it]textattack: Saving checkpoint under "checkpoints/1698255604472.ta.chkpt" at 2023-10-26 01:40:04 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 241 / 28 / 31 / 300: 100%|██████████| 300/300 [52:46<00:00, 10.55s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 241    |
| Number of failed attacks:     | 28     |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 9.33%  |
| Attack success rate:          | 89.59% |
| Average perturbed word %:     | 31.85% |
| Average num. words per input: | 8.53   |
| Avg num queries:              | 82.65  |
+-------------------------------+--------+


#### Custom Finetuned Model

In [ ]:
custom_finetuned_model, tokenizer = load_model('custom_finetuned_model.bin')
custom_finetuned_model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(custom_finetuned_model, tokenizer)

In [ ]:
# Custom attack on Custom Finetuned Model
custom_attacker = create_custom_attacker(custom_finetuned_model_wrapper, dataset, num_examples=num_examples, nr=8)
custom_attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log_8.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 5 / 0 / 5:   2%|▏         | 5/300 [00:11<11:13,  2.28s/it]textattack: Saving checkpoint under "checkpoints/1698242060508.ta.chkpt" at 2023-10-25 21:54:20 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 9 / 1 / 10:   3%|▎         | 10/300 [00:25<12:26,  2.57s/it]textattack: Saving checkpoint under "checkpoints/1698242074810.ta.chkpt" at 2023-10-25 21:54:34 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 11 / 2 / 15:   5%|▌         | 15/300 [00:33<10:29,  2.21s/it]textattack: Saving checkpoint under "checkpoints/1698242082191.ta.chkpt" at 2023-10-25 21:54:42 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 27 / 230 / 28 / 285:  95%|█████████▌| 285/300 [13:02<00:41,  2.74s/it]textattack: Saving checkpoint under "checkpoints/1698242831335.ta.chkpt" at 2023-10-25 22:07:11 after 285 attacks.


[Succeeded / Failed / Skipped / Total] 27 / 235 / 28 / 290:  97%|█████████▋| 290/300 [13:18<00:27,  2.75s/it]textattack: Saving checkpoint under "checkpoints/1698242847457.ta.chkpt" at 2023-10-25 22:07:27 after 290 attacks.


[Succeeded / Failed / Skipped / Total] 27 / 240 / 28 / 295:  98%|█████████▊| 295/300 [13:36<00:13,  2.77s/it]textattack: Saving checkpoint under "checkpoints/1698242865622.ta.chkpt" at 2023-10-25 22:07:45 after 295 attacks.


[Succeeded / Failed / Skipped / Total] 27 / 244 / 29 / 300: 100%|██████████| 300/300 [13:44<00:00,  2.75s/it]textattack: Saving checkpoint under "checkpoints/1698242874006.ta.chkpt" at 2023-10-25 22:07:54 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 27 / 244 / 29 / 300: 100%|██████████| 300/300 [13:44<00:00,  2.75s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 27     |
| Number of failed attacks:     | 244    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 81.33% |
| Attack success rate:          | 9.96%  |
| Average perturbed word %:     | 22.75% |
| Average num. words per input: | 8.53   |
| Avg num queries:              | 18.66  |
+-------------------------------+--------+


In [ ]:
# BAE attack on Custom Finetuned Model
bae_attacker = create_bae_attacker(custom_finetuned_model_wrapper, dataset, num_examples=num_examples, nr=9)
bae_attacker.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log_9.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 83 / 5 / 12 / 100:  33%|███▎      | 100/300 [15:24<30:48,  9.24s/it]textattack: Saving checkpoint under "checkpoints/1698256613208.ta.chkpt" at 2023-10-26 01:56:53 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 168 / 10 / 22 / 200:  67%|██████▋   | 200/300 [30:09<15:04,  9.05s/it]textattack: Saving checkpoint under "checkpoints/1698257499039.ta.chkpt" at 2023-10-26 02:11:39 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 254 / 17 / 29 / 300: 100%|██████████| 300/300 [45:04<00:00,  9.02s/it]textattack: Saving checkpoint under "checkpoints/1698258393632.ta.chkpt" at 2023-10-26 02:26:33 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 254 / 17 / 29 / 300: 100%|██████████| 300/300 [45:04<00:00,  9.02s/it]






+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 254    |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 5.67%  |
| Attack success rate:          | 93.73% |
| Average perturbed word %:     | 30.56% |
| Average num. words per input: | 8.53   |
| Avg num queries:              | 61.2   |
+-------------------------------+--------+


In [ ]:
# Textfooler attack on Custom Finetuned Model
textfooler_attacker = create_textfooler_attacker(custom_finetuned_model_wrapper, dataset, num_examples=num_examples, nr=10)
textfooler_attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log_10.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 81 / 7 / 12 / 100:  33%|███▎      | 100/300 [17:58<35:57, 10.79s/it]textattack: Saving checkpoint under "checkpoints/1698259474766.ta.chkpt" at 2023-10-26 02:44:34 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 165 / 13 / 22 / 200:  67%|██████▋   | 200/300 [34:51<17:25, 10.46s/it]textattack: Saving checkpoint under "checkpoints/1698260487298.ta.chkpt" at 2023-10-26 03:01:27 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 249 / 22 / 29 / 300: 100%|██████████| 300/300 [52:34<00:00, 10.52s/it]textattack: Saving checkpoint under "checkpoints/1698261550736.ta.chkpt" at 2023-10-26 03:19:10 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 249 / 22 / 29 / 300: 100%|██████████| 300/300 [52:34<00:00, 10.52s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 249    |
| Number of failed attacks:     | 22     |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 7.33%  |
| Attack success rate:          | 91.88% |
| Average perturbed word %:     | 29.4%  |
| Average num. words per input: | 8.53   |
| Avg num queries:              | 79.11  |
+-------------------------------+--------+


#### Textfooler Trained Model

In [ ]:
textfooler_trained_model, tokenizer = load_model('textfooler_trained_model.bin')
textfooler_trained_model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(textfooler_trained_model, tokenizer)

In [ ]:
# Custom attack on Textfooler Trained Model
custom_attacker = create_custom_attacker(textfooler_trained_model_wrapper, dataset, num_examples=num_examples, nr=12)
custom_attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log_12.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 5 / 0 / 5:   2%|▏         | 5/300 [00:11<10:58,  2.23s/it]textattack: Saving checkpoint under "checkpoints/1698242888113.ta.chkpt" at 2023-10-25 22:08:08 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 9 / 1 / 10:   3%|▎         | 10/300 [00:25<12:18,  2.55s/it]textattack: Saving checkpoint under "checkpoints/1698242902404.ta.chkpt" at 2023-10-25 22:08:22 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 10 / 2 / 15:   5%|▌         | 15/300 [00:32<10:10,  2.14s/it]textattack: Saving checkpoint under "checkpoints/1698242909081.ta.chkpt" at 2023-10-25 22:08:29 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 36 / 216 / 33 / 285:  95%|█████████▌| 285/300 [12:49<00:40,  2.70s/it]textattack: Saving checkpoint under "checkpoints/1698243646915.ta.chkpt" at 2023-10-25 22:20:46 after 285 attacks.


[Succeeded / Failed / Skipped / Total] 37 / 220 / 33 / 290:  97%|█████████▋| 290/300 [13:06<00:27,  2.71s/it]textattack: Saving checkpoint under "checkpoints/1698243663807.ta.chkpt" at 2023-10-25 22:21:03 after 290 attacks.


[Succeeded / Failed / Skipped / Total] 37 / 225 / 33 / 295:  98%|█████████▊| 295/300 [13:25<00:13,  2.73s/it]textattack: Saving checkpoint under "checkpoints/1698243682670.ta.chkpt" at 2023-10-25 22:21:22 after 295 attacks.


[Succeeded / Failed / Skipped / Total] 37 / 229 / 34 / 300: 100%|██████████| 300/300 [13:34<00:00,  2.71s/it]textattack: Saving checkpoint under "checkpoints/1698243691419.ta.chkpt" at 2023-10-25 22:21:31 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 37 / 229 / 34 / 300: 100%|██████████| 300/300 [13:34<00:00,  2.71s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 37     |
| Number of failed attacks:     | 229    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 76.33% |
| Attack success rate:          | 13.91% |
| Average perturbed word %:     | 23.73% |
| Average num. words per input: | 8.53   |
| Avg num queries:              | 18.01  |
+-------------------------------+--------+


In [ ]:
# BAE attack on Textfooler Trained Model
bae_attacker = create_bae_attacker(textfooler_trained_model_wrapper, dataset, num_examples=num_examples, nr=13)
bae_attacker.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log_13.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 80 / 7 / 13 / 100:  33%|███▎      | 100/300 [14:51<29:43,  8.92s/it]textattack: Saving checkpoint under "checkpoints/1698262447390.ta.chkpt" at 2023-10-26 03:34:07 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 162 / 11 / 27 / 200:  67%|██████▋   | 200/300 [29:42<14:51,  8.91s/it]textattack: Saving checkpoint under "checkpoints/1698263337837.ta.chkpt" at 2023-10-26 03:48:57 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 250 / 16 / 34 / 300: 100%|██████████| 300/300 [44:58<00:00,  9.00s/it]textattack: Saving checkpoint under "checkpoints/1698264254363.ta.chkpt" at 2023-10-26 04:04:14 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 250 / 16 / 34 / 300: 100%|██████████| 300/300 [44:58<00:00,  9.00s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 250    |
| Number of failed attacks:     | 16     |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 5.33%  |
| Attack success rate:          | 93.98% |
| Average perturbed word %:     | 33.31% |
| Average num. words per input: | 8.53   |
| Avg num queries:              | 64.07  |
+-------------------------------+--------+


In [ ]:
# Textfooler attack on Textfooler Trained Model
textfooler_attacker = create_textfooler_attacker(textfooler_trained_model_wrapper, dataset, num_examples=num_examples, nr=14)
textfooler_attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log_14.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 75 / 13 / 12 / 100:  33%|███▎      | 100/300 [20:22<40:45, 12.23s/it]textattack: Saving checkpoint under "checkpoints/1698294559042.ta.chkpt" at 2023-10-26 12:29:19 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 152 / 23 / 25 / 200:  67%|██████▋   | 200/300 [37:09<18:34, 11.15s/it]textattack: Saving checkpoint under "checkpoints/1698295565712.ta.chkpt" at 2023-10-26 12:46:05 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 235 / 31 / 34 / 300: 100%|██████████| 300/300 [54:51<00:00, 10.97s/it]textattack: Saving checkpoint under "checkpoints/1698296628062.ta.chkpt" at 2023-10-26 13:03:48 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 235 / 31 / 34 / 300: 100%|██████████| 300/300 [54:51<00:00, 10.97s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 235    |
| Number of failed attacks:     | 31     |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 10.33% |
| Attack success rate:          | 88.35% |
| Average perturbed word %:     | 28.58% |
| Average num. words per input: | 8.53   |
| Avg num queries:              | 78.24  |
+-------------------------------+--------+


## Standard Evaluation of Models

In [21]:
def test_model_on_data(model, data_loader, data_frame, batch_size, device):
    model.eval()  # Set the model to evaluation mode
    correct_predictions = 0
    total_predictions = 0
    misclassified_entries = []

    # Extract test sentences and labels from the dataframe
    test_sentences = data_frame['text'].tolist()
    test_labels = data_frame['label'].tolist()

    with torch.no_grad():
        for batch_num, batch in enumerate(tqdm(data_loader, desc='Testing', dynamic_ncols=True), 0): # Starting index at 0 for batch_num
            input_ids, attention_mask, labels = [item.to(device) for item in batch]
            outputs = model(input_ids, attention_mask=attention_mask)[0]
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

            # Identify misclassified sentences
            misclassifications = (predicted != labels).nonzero(as_tuple=True)[0].cpu().numpy()
            for idx in misclassifications:
                global_index = idx + batch_num * batch_size  # Calculate global index in test set
                misclassified_entries.append({
                    "index": global_index,
                    "sentence": test_sentences[global_index],
                    "true_label": 'offensive/hatespeech' if test_labels[global_index] == 1 else 'not offensive/hatespeech',
                    "predicted_label": 'offensive/hatespeech' if predicted[idx].item() == 1 else 'not offensive/hatespeech'
                })

    # Display test accuracy
    accuracy = 100 * correct_predictions / total_predictions
    print(f'Test Accuracy: {accuracy:.2f}%')

    # Display misclassified sentences
    #print("\nMisclassified Sentences:")
    #for entry in misclassified_entries:
    #    print(f"Index: {entry['index']}, Sentence: {entry['sentence']}, True Label: {entry['true_label']}, Predicted Label: {entry['predicted_label']}")



In [22]:
# Convert labels to tensors
train_labels_tensor = torch.tensor(train_data['label'].values, dtype=torch.long)
validation_labels_tensor = torch.tensor(validation_data['label'].values, dtype=torch.long)
test_labels_tensor = torch.tensor(test_data['label'].values, dtype=torch.long)

train_dataset = TensorDataset(train_tokens['input_ids'], train_tokens['attention_mask'], train_labels_tensor)
validation_dataset = TensorDataset(validation_tokens['input_ids'], validation_tokens['attention_mask'], validation_labels_tensor)
test_dataset = TensorDataset(test_tokens['input_ids'], test_tokens['attention_mask'], test_labels_tensor)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [23]:
initial_hate_speech_model, tokenizer = load_model('initial_hate_speech_model.bin')
custom_trained_model, tokenizer = load_model('custom_trained_model.bin')
custom_finetuned_model, tokenizer = load_model('custom_finetuned_model.bin')
textfooler_trained_model, tokenizer = load_model('textfooler_trained_model.bin')
test_model_on_data(initial_hate_speech_model, test_dataloader, test_data, BATCH_SIZE, device)

test_model_on_data(custom_trained_model, test_dataloader, test_data, BATCH_SIZE, device)

test_model_on_data(custom_finetuned_model, test_dataloader, test_data, BATCH_SIZE, device)

test_model_on_data(textfooler_trained_model, test_dataloader, test_data, BATCH_SIZE, device)

Testing: 100%|██████████| 21/21 [00:04<00:00,  4.62it/s]


Test Accuracy: 91.30%


Testing: 100%|██████████| 21/21 [00:04<00:00,  4.58it/s]


Test Accuracy: 91.46%


Testing: 100%|██████████| 21/21 [00:04<00:00,  4.83it/s]


Test Accuracy: 91.15%


Testing: 100%|██████████| 21/21 [00:04<00:00,  4.81it/s]

Test Accuracy: 90.68%
